In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from PIL import Image, ImageFile
import torchvision.models as models

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

In [3]:
data_dir = 'E:\\Projectf'
batch_size = 2  # Adjust this based on your hardware
train_dataset = datasets.ImageFolder(root=data_dir + '/train', transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(root=data_dir + '/test', transform=data_transforms['test'])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [4]:
class_names = train_dataset.classes

In [5]:
model = models.resnet18(pretrained=True)

# Modify the classifier for your specific task
num_classes = len(class_names)
# Define the classifier as a Sequential block
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),  # Add an additional fully connected layer
    nn.ReLU(),                             # Add a ReLU activation
    nn.Dropout(0.5),                       # Add dropout for regularization
    nn.Linear(512, num_classes)            # Final fully connected layer for classification
)

C:\Users\USERROR\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\USERROR\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [7]:
num_epochs = 10  # Adjust the number of epochs as needed
best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels.data)
    
    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = correct.double() / len(train_loader.dataset)
    
    # Validation (test) phase
    model.eval()
    test_correct = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            test_correct += torch.sum(preds == labels.data)
    
    test_accuracy = test_correct.double() / len(test_loader.dataset)
    
    print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.4f} | Test Accuracy: {test_accuracy:.4f}')
    
    # Save the model if it has the best test accuracy so far
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.pth')
        best_accuracy = test_accuracy

print('Training complete. Best test accuracy:', best_accuracy)

Epoch 1/10 | Train Loss: 1.5573 | Train Accuracy: 0.2500 | Test Accuracy: 0.2857
Epoch 2/10 | Train Loss: 1.4317 | Train Accuracy: 0.3088 | Test Accuracy: 0.3810
Epoch 3/10 | Train Loss: 1.2963 | Train Accuracy: 0.3824 | Test Accuracy: 0.7143
Epoch 4/10 | Train Loss: 1.2945 | Train Accuracy: 0.3824 | Test Accuracy: 0.5476
Epoch 5/10 | Train Loss: 1.2531 | Train Accuracy: 0.4265 | Test Accuracy: 0.7143
Epoch 6/10 | Train Loss: 1.1046 | Train Accuracy: 0.5882 | Test Accuracy: 0.6190
Epoch 7/10 | Train Loss: 0.9740 | Train Accuracy: 0.6029 | Test Accuracy: 0.6429
Epoch 8/10 | Train Loss: 0.9643 | Train Accuracy: 0.6324 | Test Accuracy: 0.7857
Epoch 9/10 | Train Loss: 1.0181 | Train Accuracy: 0.5882 | Test Accuracy: 0.8571
Epoch 10/10 | Train Loss: 0.8493 | Train Accuracy: 0.6471 | Test Accuracy: 0.8095
Training complete. Best test accuracy: tensor(0.8571, dtype=torch.float64)
